# 🧑‍💻 Notebookを使って分析してみよう
Snowflake Notebooksはデータサイエンス、ML、およびデータエンジニアリングのワークロードで、Snowflakeデータをこれまで以上に迅速かつ容易に活用できるようにする、統一されたセルベースの対話型開発環境です。

セルは次の言語をサポートしています: Python (デフォルト)、SQL、Markdown (メモ)

## ✅ Step1 - ワークシートで取得したデータをNotebookで確認
Notebookではワークシートで取得したデータを、SQLを使用して取得することができます。今回はワークシートで作成したビュー (DAILY_WEATHER_VIEW)から2022年2月のドイツのハンブルクの降水量と最大瞬間風速を取得します。


In [ ]:
SELECT
    dw.country_desc,
    dw.city_name,
    dw.date_valid_std,
    MAX(dw.max_wind_speed_100m_mph) AS max_wind_speed_100m_mph,
    AVG(dw.tot_precipitation_in) AS tot_precipitation_in
FROM harmonized.daily_weather_v dw
WHERE 1=1
    AND dw.country_desc IN ('Germany')
    AND dw.city_name = 'Hamburg'
    AND YEAR(date_valid_std) = '2022'
    AND MONTH(date_valid_std) = '2' -- 2月
GROUP BY dw.country_desc, dw.city_name, dw.date_valid_std
ORDER BY dw.date_valid_std ASC;

## ✅ Step2 - Pythonで分析対象になるテーブルを準備
上記でマーケットプレイスから取得したデータ使って作成したテーブルに対して、ドイツのハンブルクでの2022年2月の売上高データを追加

In [ ]:
SELECT
    fd.date_valid_std AS date,
    fd.city_name,
    fd.country_desc,
    ZEROIFNULL(SUM(odv.price)) AS daily_sales,
    ROUND(AVG(fd.avg_temperature_air_2m_f),2) AS avg_temperature_fahrenheit,
    ROUND(AVG(demo_db.analytics.fahrenheit_to_celsius(fd.avg_temperature_air_2m_f)),2) AS avg_temperature_celsius,
    ROUND(AVG(fd.tot_precipitation_in),2) AS avg_precipitation_inches,
    ROUND(AVG(demo_db.analytics.inch_to_millimeter(fd.tot_precipitation_in)),2) AS avg_precipitation_millimeters,
    MAX(fd.max_wind_speed_100m_mph) AS max_wind_speed_100m_mph
FROM harmonized.daily_weather_v fd
LEFT JOIN harmonized.orders_v odv
    ON fd.date_valid_std = DATE(odv.order_ts)
    AND fd.city_name = odv.primary_city
    AND fd.country_desc = odv.country
WHERE 1=1
    AND fd.country_desc = 'Germany'
    AND fd.city = 'Hamburg'
    AND fd.yyyy_mm = '2022-02'
GROUP BY fd.date_valid_std, fd.city_name, fd.country_desc
ORDER BY fd.date_valid_std ASC;

## ✅ Step3 - SQLで作成したテーブルをPythonのPandasに簡単に変換
ワークシートやNotebook上でSQLで作成したテーブルを、PythonのPandas型に簡単に変換することができます。

In [ ]:
df = cell3.to_pandas()

df.head()

## ✅ Step4 - Pythonを使用して準備したデータを簡単に可視化
ワークシートやNotebookで集計したデータをPythonを使って簡単に分析できます。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib as mpl
from snowflake.snowpark.context import get_active_session

session = get_active_session()

# データ前処理 ----------------------------------------------------------
df.columns = df.columns.str.strip().str.lower()
df = df.loc[:, ~df.columns.duplicated()]  # 重複した列の処理
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")
df["daily_sales"] = df["daily_sales"].round(-6) # 100万$単位

# 背景スタイル----------------------------------------------------------
for style in ("seaborn-v0_8-whitegrid", "seaborn-whitegrid", "ggplot"):
    if style in mpl.style.available:
        plt.style.use(style); break

plt.rcParams.update({
    "font.size": 9,
    "axes.grid": False,                
    "axes.spines.right": False,        
})

fig, ax_sales = plt.subplots(figsize=(10, 5))
fig.subplots_adjust(right=0.84)         
ax_sales.set_facecolor("white"); fig.patch.set_facecolor("white")

# 売上（左軸）----------------------------------------------------------
ax_sales.plot(df["date"], df["daily_sales"],
              label="Daily Sales ($)",
              color="#1f77b4", lw=2, marker="o", ms=4, zorder=4)
ax_sales.set_ylabel("Daily Sales ($)")
ax_sales.yaxis.set_major_formatter(FuncFormatter(lambda x,_: f"{x/1e6:.0f} M"))
ax_sales.tick_params(axis="y", labelcolor="#1f77b4")

ax_sales.grid(axis="y", linestyle="--", lw=0.4, color="grey", alpha=0.5, zorder=0)

# 降水量（右：内側） -----------------------------------------------------
ax_prec = ax_sales.twinx()
ax_prec.spines.right.set_position(("axes", 1.0)) 
ax_prec.bar(df["date"], df["avg_precipitation_millimeters"],
            width=0.8, color="#177c1c", alpha=0.35,
            label="Precipitation (mm)", zorder=1)
ax_prec.set_ylabel("Precipitation (mm)")
ax_prec.tick_params(axis="y", labelcolor="#177c1c", pad=4)
# 副軸スパイン＆グリッドを消去
ax_prec.spines["right"].set_visible(False)
ax_prec.grid(False)

# 風速（右：外側 2 本目） ---------------------------------------------------
ax_wind = ax_sales.twinx()
ax_wind.spines.right.set_position(("axes", 1.08)) 
ax_wind.plot(df["date"], df["max_wind_speed_100m_mph"],
             label="Max Wind Speed (mph)",
             color="#d62728", lw=1.6, ls="--", marker="x", ms=5, zorder=3)
ax_wind.set_ylabel("Wind Speed (mph)")
ax_wind.tick_params(axis="y", labelcolor="#d62728")

ax_wind.spines["right"].set_color("#d62728")
ax_wind.grid(False)

# 凡例 ----------------------------------------------------------------------
lines, labels = [], []
for ax in (ax_sales, ax_prec, ax_wind):
    l, lab = ax.get_legend_handles_labels()
    lines += l; labels += lab
leg = ax_sales.legend(lines, labels, loc="upper left",
                      frameon=True, framealpha=1,
                      bbox_to_anchor=(0.02, 0.98))
leg.get_frame().set_facecolor("white")
leg.get_frame().set_edgecolor("none")

# X 軸：日付ラベル ------------------------------------------------------------
ax_sales.set_xlim(df["date"].min(), df["date"].max())  
ax_sales.tick_params(axis="x", rotation=45, pad=3)      

# タイトル -------------------------------------------------------------------
plt.title("Weather vs Food-truck Sales — Hamburg, Feb 2022", weight="bold")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

cols = ['daily_sales','avg_precipitation_millimeters','max_wind_speed_100m_mph']

# 画面のレイアウト整形
plt.rcParams.update({'font.size': 8})
fig, (ax_hm, ax_sc) = plt.subplots(
    1, 2, figsize=(8, 3.5), dpi=110   # 1 行 2 列レイアウト
)

# 1.相関ヒートマップ ------------------------------------------------------
corr = df[cols].corr()
im = ax_hm.imshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
ax_hm.set_xticks(range(len(cols))); ax_hm.set_yticks(range(len(cols)))
ax_hm.set_xticklabels(cols, rotation=45, ha='right')
ax_hm.set_yticklabels(cols)
for i in range(len(cols)):
    for j in range(len(cols)):
        ax_hm.text(j, i, f'{corr.iloc[i, j]:.2f}',
                   ha='center', va='center', color='black', fontsize=7)
ax_hm.set_title('Correlation')

# 2. 重回帰：実測 vs 予測 -------------------------------------------------
X = df[['avg_precipitation_millimeters', 'max_wind_speed_100m_mph']]
y = df['daily_sales']
model = LinearRegression().fit(X, y)
y_pred = model.predict(X)

ax_sc.scatter(y, y_pred, s=12, alpha=0.7)
lim = [min(y.min(), y_pred.min()), max(y.max(), y_pred.max())]
ax_sc.plot(lim, lim, 'k--', lw=0.8)
ax_sc.set_xlabel('Actual ($)'); ax_sc.set_ylabel('Predicted ($)')
ax_sc.set_title(f'Actual vs Predicted (R² = {model.score(X, y):.2f})')

fig.colorbar(im, ax=ax_hm, fraction=0.046)

plt.tight_layout()
plt.show()

print('Coefficients ($/unit):',
      {c: round(v) for c, v in zip(X.columns, model.coef_)})
print('Intercept ($):', round(model.intercept_))